In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets list -s waste


ref                                                              title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
techsash/waste-classification-data                               Waste Classification data                            447914276  2019-06-16 03:24:52.733000          49971        537  0.875            
joebeachcapital/food-waste                                       Food Waste                                                7157  2023-08-03 01:36:24.530000           7391         88  1.0              
alistairking/recyclable-and-household-waste-classification       Recyclable and Household Waste Classification        964682710  2024-05-18 19:06:35.757000          11563        162  1.0          

In [ ]:
!kaggle datasets download -d alistairking/recyclable-and-household-waste-classification
!unzip recyclable-and-household-waste-classification.zip -d waste_dataset


Streaming output truncated to the last 5000 lines.
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_1.png  
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_10.png  
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_100.png  
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_101.png  
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_102.png  
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_103.png  
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_104.png  
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_105.png  
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_106.png  
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_107.png  
  inflating: waste_dataset/images/images/plastic_shopping_bags/default/Image_108.p

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Path where your dataset was unzipped
data_dir = "/content/waste_dataset/images/images"


# List folders (should show 'Recyclable' and 'Household' or similar)
print("Folders:", os.listdir(data_dir))

# Count number of images per class
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        print(f"{folder}: {len(os.listdir(folder_path))} images")


Folders: ['aluminum_soda_cans', 'plastic_shopping_bags', 'cardboard_boxes', 'plastic_food_containers', 'plastic_soda_bottles', 'styrofoam_cups', 'magazines', 'aerosol_cans', 'aluminum_food_cans', 'styrofoam_food_containers', 'plastic_detergent_bottles', 'newspaper', 'steel_food_cans', 'plastic_water_bottles', 'glass_cosmetic_containers', 'plastic_straws', 'tea_bags', 'plastic_cup_lids', 'eggshells', 'clothing', 'glass_food_jars', 'shoes', 'disposable_plastic_cutlery', 'cardboard_packaging', 'plastic_trash_bags', 'paper_cups', 'glass_beverage_bottles', 'food_waste', 'coffee_grounds', 'office_paper']
aluminum_soda_cans: 2 images
plastic_shopping_bags: 2 images
cardboard_boxes: 2 images
plastic_food_containers: 2 images
plastic_soda_bottles: 2 images
styrofoam_cups: 2 images
magazines: 2 images
aerosol_cans: 2 images
aluminum_food_cans: 2 images
styrofoam_food_containers: 2 images
plastic_detergent_bottles: 2 images
newspaper: 2 images
steel_food_cans: 2 images
plastic_water_bottles: 2 im

In [ ]:
!unzip recyclable-and-household-waste-classification.zip -d /content/full_dataset
!find /content/full_dataset -type d -maxdepth 3


Streaming output truncated to the last 5000 lines.
  inflating: /content/full_dataset/images/images/plastic_shopping_bags/default/Image_13.png  
  inflating: /content/full_dataset/images/images/plastic_shopping_bags/default/Image_130.png  
  inflating: /content/full_dataset/images/images/plastic_shopping_bags/default/Image_131.png  
  inflating: /content/full_dataset/images/images/plastic_shopping_bags/default/Image_132.png  
  inflating: /content/full_dataset/images/images/plastic_shopping_bags/default/Image_133.png  
  inflating: /content/full_dataset/images/images/plastic_shopping_bags/default/Image_134.png  
  inflating: /content/full_dataset/images/images/plastic_shopping_bags/default/Image_135.png  
  inflating: /content/full_dataset/images/images/plastic_shopping_bags/default/Image_136.png  
  inflating: /content/full_dataset/images/images/plastic_shopping_bags/default/Image_137.png  
  inflating: /content/full_dataset/images/images/plastic_shopping_bags/default/Image_138.png  


In [ ]:
import os

data_dir = "/content/full_dataset/images"

count = 0
for root, dirs, files in os.walk(data_dir):
    count += len([f for f in files if f.lower().endswith(('.jpg', '.png', '.jpeg'))])

print("Total images found:", count)


Total images found: 15000


In [ ]:
import os
from sklearn.model_selection import train_test_split
import shutil

base_dir = '/content/waste_data_split'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

classes = os.listdir(data_dir)
print("Classes:", classes)

for cls in classes:
    cls_path = os.path.join(data_dir, cls)
    if os.path.isdir(cls_path):
        images = os.listdir(cls_path)
        train_imgs, test_imgs = train_test_split(images, test_size=0.2, random_state=42)

        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

        for img in train_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(train_dir, cls, img))
        for img in test_imgs:
            shutil.copy(os.path.join(cls_path, img), os.path.join(test_dir, cls, img))

print("✅ Dataset successfully split into train and test folders!")


Classes: ['images']


IsADirectoryError: [Errno 21] Is a directory: '/content/full_dataset/images/images/coffee_grounds'

In [ ]:
!find /content/full_dataset -type d -maxdepth 4


find: warning: you have specified the global option -maxdepth after the argument -type, but global options are not positional, i.e., -maxdepth affects tests specified before it as well as those specified after it.  Please specify global options before other arguments.
/content/full_dataset
/content/full_dataset/images
/content/full_dataset/images/images
/content/full_dataset/images/images/aluminum_soda_cans
/content/full_dataset/images/images/aluminum_soda_cans/real_world
/content/full_dataset/images/images/aluminum_soda_cans/default
/content/full_dataset/images/images/plastic_shopping_bags
/content/full_dataset/images/images/plastic_shopping_bags/real_world
/content/full_dataset/images/images/plastic_shopping_bags/default
/content/full_dataset/images/images/cardboard_boxes
/content/full_dataset/images/images/cardboard_boxes/real_world
/content/full_dataset/images/images/cardboard_boxes/default
/content/full_dataset/images/images/plastic_food_containers
/content/full_dataset/images/ima

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Correct base dataset path
data_dir = "/content/full_dataset/images/images"

# Split directories
base_dir = '/content/waste_data_split'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# All class folders
classes = os.listdir(data_dir)
print("Classes found:", classes)

for cls in classes:
    cls_path = os.path.join(data_dir, cls)
    if os.path.isdir(cls_path):
        # Collect images from both 'real_world' and 'default' subfolders
        all_images = []
        for subfolder in ['real_world', 'default']:
            sub_path = os.path.join(cls_path, subfolder)
            if os.path.exists(sub_path):
                all_images.extend([os.path.join(sub_path, img) for img in os.listdir(sub_path)])

        # Split into train/test
        train_imgs, test_imgs = train_test_split(all_images, test_size=0.2, random_state=42)

        # Create destination folders
        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

        # Copy images
        for img in train_imgs:
            shutil.copy(img, os.path.join(train_dir, cls, os.path.basename(img)))
        for img in test_imgs:
            shutil.copy(img, os.path.join(test_dir, cls, os.path.basename(img)))

print("✅ Dataset successfully combined and split into train/test sets!")

# 🔹 Now create TensorFlow datasets AFTER copying
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir, image_size=(180, 180), batch_size=32)
test_ds = tf.keras.utils.image_dataset_from_directory(test_dir, image_size=(180, 180), batch_size=32)


Classes found: ['aluminum_soda_cans', 'plastic_shopping_bags', 'cardboard_boxes', 'plastic_food_containers', 'plastic_soda_bottles', 'styrofoam_cups', 'magazines', 'aerosol_cans', 'aluminum_food_cans', 'styrofoam_food_containers', 'plastic_detergent_bottles', 'newspaper', 'steel_food_cans', 'plastic_water_bottles', 'glass_cosmetic_containers', 'plastic_straws', 'tea_bags', 'plastic_cup_lids', 'eggshells', 'clothing', 'glass_food_jars', 'shoes', 'disposable_plastic_cutlery', 'cardboard_packaging', 'plastic_trash_bags', 'paper_cups', 'glass_beverage_bottles', 'food_waste', 'coffee_grounds', 'office_paper']
✅ Dataset successfully combined and split into train/test sets!
Found 7260 files belonging to 31 classes.
Found 2760 files belonging to 31 classes.


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Original dataset
data_dir = "/content/full_dataset/images/images"

# Drive save path
drive_base_dir = "/content/drive/MyDrive/waste_data_split"
train_dir = os.path.join(drive_base_dir, 'train')
test_dir = os.path.join(drive_base_dir, 'test')

# Make sure folders exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Rebuild test split from source
classes = os.listdir(data_dir)
print("Rebuilding test split for classes:", len(classes))

for cls in classes:
    cls_path = os.path.join(data_dir, cls)
    if os.path.isdir(cls_path):
        # Collect all images from both subfolders
        all_images = []
        for sub in ["real_world", "default"]:
            sub_path = os.path.join(cls_path, sub)
            if os.path.exists(sub_path):
                all_images.extend([os.path.join(sub_path, img) for img in os.listdir(sub_path)])

        # Skip empty classes
        if len(all_images) == 0:
            continue

        # Split (20% test)
        _, test_imgs = train_test_split(all_images, test_size=0.2, random_state=42)

        # Create test folder in Drive
        os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

        # Copy test images
        for img in test_imgs:
            shutil.copy(img, os.path.join(test_dir, cls, os.path.basename(img)))

print("\n✅ Test dataset successfully rebuilt and saved to Google Drive!")


Rebuilding test split for classes: 30

✅ Test dataset successfully rebuilt and saved to Google Drive!


In [ ]:
!ls /content/drive/MyDrive/waste_data_split


test  train


In [ ]:
import os

data_dir = "/content/waste_dataset/images"
files = os.listdir(data_dir)

print("Total files:", len(files))
print("First 10 files:", files[:10])


Total files: 1
First 10 files: ['images']
